In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
import time
import pandas as pd
from tqdm import tqdm

In [3]:
import numpy as np

In [4]:
from selenium.webdriver.common.by import By

In [6]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [5]:
path_CDM = './chrome_driver/chromedriver.exe'

In [20]:
info = {} #словарь, где ключ - имя преподавателя, а значение -  датафрейм с сырой информацией


In [89]:
try:
    url = 'http://wikimipt.org/wiki/'
    browser = webdriver.Chrome(path_CDM)
    browser.get(url)
    time.sleep(1)
    
    kafs = browser.find_elements(By.XPATH, '/html/body/div[3]/div[2]/div[4]/div[3]/ul/li') #находим количество кафедр
    
    num_kafs = len(kafs)
    
    for i in range(1, num_kafs+1): # итерируемся по кафедрам
        
        kaf = browser.find_element(By.XPATH, '/html/body/div[3]/div[2]/div[4]/div[3]/ul/li[' + str(i) + ']/a')
        kaf.click()  #проваливаемся в кафедру
        
        preps = browser.find_elements_by_xpath('/html/body/div[3]/div[2]/div[4]/div[3]/ul/li') #находим количество преподавателей
        num_preps = len(preps)

        for j in range(1, num_preps+1): #итерируемся по преподавателям
            prep = browser.find_element(By.XPATH, '/html/body/div[3]/div[2]/div[4]/div[3]/ul/li[' + str(j) + ']/a')

            prep.click() #проваливаемся на страничку преподавателя
            #считываем таблицу со страницы с информацией о преподавателе
            
            df = pd.read_html(browser.find_element_by_tag_name("table").get_attribute('outerHTML'))[0].T 
            df = df.rename(columns=df.iloc[0]).drop(df.index[0])
            
            #добавляем информацию в словарь
            info[df.iloc[0,0]] = df

            browser.back() #возвращаемся на страницу кафедры
        browser.back() #возвращаемся к списку кафедр
            
            
    
   
    
except Exception as e:
    print('Ошибка')
    print(e)
    
finally:
    time.sleep(3)  
    browser.quit()  
    print('Закрыл')

C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  
C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


Ошибка
Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=96.0.4664.93)
Stacktrace:
Backtrace:
	Ordinal0 [0x007E6903+2517251]
	Ordinal0 [0x0077F8E1+2095329]
	Ordinal0 [0x00682848+1058888]
	Ordinal0 [0x0066BF90+966544]
	Ordinal0 [0x006C8999+1345945]
	Ordinal0 [0x006D5342+1397570]
	Ordinal0 [0x006C639B+1336219]
	Ordinal0 [0x006A27A7+1189799]
	Ordinal0 [0x006A3609+1193481]
	GetHandleVerifier [0x00975904+1577972]
	GetHandleVerifier [0x00A20B97+2279047]
	GetHandleVerifier [0x00876D09+534521]
	GetHandleVerifier [0x00875DB9+530601]
	Ordinal0 [0x00784FF9+2117625]
	Ordinal0 [0x007898A8+2136232]
	Ordinal0 [0x007899E2+2136546]
	Ordinal0 [0x00793541+2176321]
	BaseThreadInitThunk [0x750FFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77287A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77287A6E+238]

Закрыл


In [44]:
len(info.values()) #количество преподавателей


737

In [64]:
def string_to_mark(str):  #костыльная функция для перевода строки с информацией об оценке преподавателя в оценку   
    if str[5] == '(':
        return np.nan  #нет оценок
    else:
        if str[6] == ' ':
            return str[5] #оценка целая
        return str[5:9] #оценка дробная
    
    

In [66]:
string_to_mark('012344.86 ( 51 голос )')

'4.86'

In [74]:
final = np.array([])
#бежим по словарю с преподавателями и составляем датасет
for x in info.values():
    prep = [np.nan]*9
    
    prep[0] = x.iloc[0][0]  #имя преподавателя
    
    if 'Дата рождения' in x.columns:
        prep[1] = x['Дата рождения'][1]
        
    prep[2] = x['Работает'][1]
    if 'Учёная степень' in x.columns:
        prep[3] = x['Учёная степень'][1]
    prep[4] = string_to_mark(x['Знания'][1])
    prep[5] = string_to_mark(x['Умение преподавать'][1])
    prep[6] = string_to_mark(x['В общении'][1])
    prep[7] = string_to_mark(x['«Халявность»'][1])
    prep[8] = string_to_mark(x['Общая оценка'][1])
    final = np.append(final, prep)
    


In [78]:
final_df = pd.DataFrame(final.reshape((737,-1)))

In [87]:
#сохраняем csv
final_df.to_csv('preps.csv')


In [88]:
pd.read_csv('preps.csv').head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8
0,0,Агаханов Назар Хангельдыевич,25 июня 1954,Кафедра высшей математики,Кандидат физико-математических наук (1988),4.86,4.77,4.79,4.25,4.79
1,1,Агаханова Яна Сергеевна,NaN,Кафедра высшей математики,Кандидат физико-математических наук (2007),4.51,4.63,4.85,4.81,4.81
2,2,Алексеев Василий Антонович,NaN,Кафедра высшей математики Кафедра информатики,NaN,4.67,4.33,4.50,4.17,4.67
3,3,Амосов Григорий Геннадьевич,27 ноября 1972,Кафедра высшей математики,Доктор физико-математических наук,4.42,3.42,4.33,4.44,4.00
4,4,Андреев Михаил Юрьевич,NaN,Кафедра высшей математики,кандидат физико-математических наук,3.46,3.50,3.73,3.47,3.69
